In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import cv2
import yaml
import random

In [ ]:
def imgResize(img, x1, y1, x2, y2, width, height):
    originalHeight, originalWidth = img.shape[:2]
    scaleWidth = width / originalWidth
    scaleHeight = height / originalHeight
    imgResized = cv2.resize(
        img, (width, height), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * scaleWidth), int(y1 * scaleHeight)
    x2New, y2New = int(x2 * scaleWidth), int(y2 * scaleHeight)
    return imgResized, x1New, y1New, x2New, y2New


In [ ]:
def LisaToYolo(dataPath):
    filePathAnnot = dataPath+"/annotations.csv"
    dfAnnot = pd.read_csv(filePathAnnot, sep=',')
    identicalSigns = dfAnnot['class'].unique().tolist()
    identicalFiles = dfAnnot['filename'].unique()
    for file in identicalFiles:
        fileImg = os.path.join(dataPath, file)
        if not os.path.exists(fileImg):
            continue
        fileDataFrame = dfAnnot[dfAnnot['filename'] == file]
        fileAnnotations = []
        for index, row in fileDataFrame.iterrows():
            img = cv2.imread(dataPath+"/"+row[0])
            imgResized, x1, y1, x2, y2 = imgResize(img, row[1], row[2], row[3], row[4], 640, 640)
            heigh, wid, _ = img.shape
            xCenter = ((x1 + x2) // 2) / wid
            yCenter = ((y1 + y2) // 2) / heigh
            width = (x2 - x1) / wid
            height = (y2 - y1) / heigh
            idClass = identicalSigns.index(row[5])
            cv2.imwrite("./Yolov5newdata"+"/"+file.split("/")[-1], imgResized)
            fileAnnotations.append(f"{idClass} {xCenter} {yCenter} {width} {height}")
        with open(os.path.join('./Yolov5newdata', f"{Path(file).stem}.txt"), "w") as f:
            f.write("\n".join(fileAnnotations))

In [ ]:
def trainValCreation(dataPath):
    filePathAnnot = "./Data/annotations.csv"
    dfAnnot = pd.read_csv(filePathAnnot, sep=',')
    identicalSigns = dfAnnot['class'].unique().tolist()
    imgFiles = [dataPath+"/"+f for f in os.listdir(dataPath) if f.endswith('.jpg') or f.endswith('.png')]
    random.shuffle(imgFiles)
    splitRatio = 0.8
    trainFiles = imgFiles[:int(len(imgFiles) * splitRatio)]
    valFiles = imgFiles[int(len(imgFiles) * splitRatio):]

    with open("train.txt", "w") as f:
        f.writelines([ file + "\n" for file in trainFiles])

    with open("val.txt", "w") as f:
        f.writelines([file + "\n" for file in valFiles])
    return len(identicalSigns), identicalSigns


In [ ]:

def yamlCreation(file, noClass, listClass):
    data = {
    'train': './train.txt',
    'val': './val.txt',
    'nc': noClass,
    'names': listClass
    }

    with open(file, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

In [ ]:
os.mkdir('./Yolov5newdata')

In [ ]:
LisaToYolo('./DataSet2')
noClass, listClass = trainValCreation('./Yolov5newdata')
yamlCreation('dataset.yaml', noClass, listClass)